### Nathan Reed
### Project Milestone 3
### Prof. Williams
### 07/16/2023

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Importing Data from Website

In [2]:
years = list(range(2010, 2024))
start_html = "https://www.basketball-reference.com/leagues/NBA_{}_totals.html"
for year in years:
    html = start_html.format(year)
    data = requests.get(html)
    
    with open("npaplayers{}.html".format(year), "w+", encoding="utf-8") as f:
        f.write(data.text)

In [3]:
dfs = []
for year in years:
    with open("npaplayers{}.html".format(year), encoding="utf-8") as f:
        page = f.read()
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr').decompose()
    player_table = soup.find(id="div_totals_stats")
    player = pd.read_html(str(player_table))[0]
    
    dfs.append(player)

In [4]:
players_df = pd.concat(dfs)
players_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1,Arron Afflalo,SG,24,DEN,82,75,2221,272,585,...,.735,59,193,252,138,46,30,74,225,724
1,2,Alexis Ajinça,C,21,CHA,6,0,30,5,10,...,.000,1,3,4,0,1,1,2,5,10
2,3,LaMarcus Aldridge,PF,24,POR,78,78,2922,579,1169,...,.757,192,435,627,160,67,48,104,231,1393
3,4,Joe Alexander,SF,23,CHI,8,0,29,1,6,...,.667,2,3,5,2,1,1,0,9,4
4,5,Malik Allen,PF,31,DEN,51,3,456,46,116,...,.923,36,46,82,16,11,5,22,68,105


# Data Cleaning

## Replacing All Column Headers

In [5]:
players_df.columns =["Rank", "Player", "Position",  "Age", "Team", "Game", "Games Started", "Minutes Played", 
                                "Field Goals", "Field Goals Attempted", "Field Goal Percentage", "Three Point Field Goals", 
                                "Three Pointers Attempted", "Three Point Percentage", "Two Point Field Goals", 
                                "Two Pointers Attempted", "Two Point Percentage", "Effective Field Goal Percentage", 
                                "Free Throws Made", "Free Throws Attempted", "Free Throw Percentage", "Offensive Rebounds",
                                "Defensive Rebounds", "Total Rebounds", "Assists", "Steals", "Blocks", "Turnovers", 
                                "Personal Fouls", "Points"]
players_df.head()

,Rank,Player,Position,Age,Team,Game,Games Started,Minutes Played,Field Goals,Field Goals Attempted,...,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,1,Arron Afflalo,SG,24,DEN,82,75,2221,272,585,...,.735,59,193,252,138,46,30,74,225,724
1,2,Alexis Ajinça,C,21,CHA,6,0,30,5,10,...,.000,1,3,4,0,1,1,2,5,10
2,3,LaMarcus Aldridge,PF,24,POR,78,78,2922,579,1169,...,.757,192,435,627,160,67,48,104,231,1393
3,4,Joe Alexander,SF,23,CHI,8,0,29,1,6,...,.667,2,3,5,2,1,1,0,9,4
4,5,Malik Allen,PF,31,DEN,51,3,456,46,116,...,.923,36,46,82,16,11,5,22,68,105


When I scraped the code from online the headers did not come with it. I replaced the columns with the names that were supposed to be the headers.

## Drop the Rank Column

In [6]:
players_df1 = players_df.drop(players_df.columns[0],axis=1)
players_df1.head()

,Player,Position,Age,Team,Game,Games Started,Minutes Played,Field Goals,Field Goals Attempted,Field Goal Percentage,...,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,Arron Afflalo,SG,24,DEN,82,75,2221,272,585,.465,...,.735,59,193,252,138,46,30,74,225,724
1,Alexis Ajinça,C,21,CHA,6,0,30,5,10,.500,...,.000,1,3,4,0,1,1,2,5,10
2,LaMarcus Aldridge,PF,24,POR,78,78,2922,579,1169,.495,...,.757,192,435,627,160,67,48,104,231,1393
3,Joe Alexander,SF,23,CHI,8,0,29,1,6,.167,...,.667,2,3,5,2,1,1,0,9,4
4,Malik Allen,PF,31,DEN,51,3,456,46,116,.397,...,.923,36,46,82,16,11,5,22,68,105


This column did not make a lot of sense as the index column works just as well at labeling all the rows. There was also a few of each rank which makes it not unique or useful.

## Get Rid of Rows for Invalid Data

In [7]:
players_df1.head(26)

,Player,Position,Age,Team,Game,Games Started,Minutes Played,Field Goals,Field Goals Attempted,Field Goal Percentage,...,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,Arron Afflalo,SG,24,DEN,82,75,2221,272,585,.465,...,.735,59,193,252,138,46,30,74,225,724
1,Alexis Ajinça,C,21,CHA,6,0,30,5,10,.500,...,.000,1,3,4,0,1,1,2,5,10
2,LaMarcus Aldridge,PF,24,POR,78,78,2922,579,1169,.495,...,.757,192,435,627,160,67,48,104,231,1393
3,Joe Alexander,SF,23,CHI,8,0,29,1,6,.167,...,.667,2,3,5,2,1,1,0,9,4
4,Malik Allen,PF,31,DEN,51,3,456,46,116,.397,...,.923,36,46,82,16,11,5,22,68,105
5,Ray Allen*,SG,34,BOS,80,80,2819,464,973,.477,...,.913,45,208,253,210,64,25,129,182,1304
6,Tony Allen,SG,28,BOS,54,8,889,129,253,.510,...,.605,52,93,145,72,59,19,66,109,330
7,Rafer Alston,PG,33,TOT,52,38,1421,155,446,.348,...,.728,13,116,129,176,50,10,95,103,427
8,Rafer Alston,PG,33,NJN,27,13,766,95,277,.343,...,.815,9,66,75,104,27,5,61,50,262
9,Rafer Alston,PG,33,MIA,25,25,655,60,169,.355,...,.556,4,50,54,72,23,5,34,53,165


In [8]:
players_df2 = players_df1[players_df1["Player"] != "Player"]
players_df2.head(26)

,Player,Position,Age,Team,Game,Games Started,Minutes Played,Field Goals,Field Goals Attempted,Field Goal Percentage,...,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,Arron Afflalo,SG,24,DEN,82,75,2221,272,585,.465,...,.735,59,193,252,138,46,30,74,225,724
1,Alexis Ajinça,C,21,CHA,6,0,30,5,10,.500,...,.000,1,3,4,0,1,1,2,5,10
2,LaMarcus Aldridge,PF,24,POR,78,78,2922,579,1169,.495,...,.757,192,435,627,160,67,48,104,231,1393
3,Joe Alexander,SF,23,CHI,8,0,29,1,6,.167,...,.667,2,3,5,2,1,1,0,9,4
4,Malik Allen,PF,31,DEN,51,3,456,46,116,.397,...,.923,36,46,82,16,11,5,22,68,105
5,Ray Allen*,SG,34,BOS,80,80,2819,464,973,.477,...,.913,45,208,253,210,64,25,129,182,1304
6,Tony Allen,SG,28,BOS,54,8,889,129,253,.510,...,.605,52,93,145,72,59,19,66,109,330
7,Rafer Alston,PG,33,TOT,52,38,1421,155,446,.348,...,.728,13,116,129,176,50,10,95,103,427
8,Rafer Alston,PG,33,NJN,27,13,766,95,277,.343,...,.815,9,66,75,104,27,5,61,50,262
9,Rafer Alston,PG,33,MIA,25,25,655,60,169,.355,...,.556,4,50,54,72,23,5,34,53,165


## Reset Index Column

In [9]:
players_df3 = players_df2.reset_index(drop=True)
players_df3.tail()

,Player,Position,Age,Team,Game,Games Started,Minutes Played,Field Goals,Field Goals Attempted,Field Goal Percentage,...,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
8976,Thaddeus Young,PF,34,TOR,54,9,795,108,198,.545,...,.692,71,95,166,75,54,5,42,88,240
8977,Trae Young,PG,24,ATL,73,73,2541,597,1390,.429,...,.886,56,161,217,741,80,9,300,104,1914
8978,Omer Yurtseven,C,24,MIA,9,0,83,16,27,.593,...,.833,8,15,23,2,2,2,4,16,40
8979,Cody Zeller,C,30,MIA,15,2,217,37,59,.627,...,.686,25,39,64,10,3,4,14,33,98
8980,Ivica Zubac,C,25,LAC,76,76,2170,326,514,.634,...,.697,236,520,756,77,29,98,117,219,818


I reset the index so that it is different for each one. This made it so that it actually reflected the correct amount of rows.

## Missing Values

In [10]:
nan_count = players_df3.isna().sum()
print(nan_count )

Player                               0
Position                             0
Age                                  0
Team                                 0
Game                                 0
Games Started                        0
Minutes Played                       0
Field Goals                          0
Field Goals Attempted                0
Field Goal Percentage               56
Three Point Field Goals              0
Three Pointers Attempted             0
Three Point Percentage             955
Two Point Field Goals                0
Two Pointers Attempted               0
Two Point Percentage               119
Effective Field Goal Percentage     56
Free Throws Made                     0
Free Throws Attempted                0
Free Throw Percentage              499
Offensive Rebounds                   0
Defensive Rebounds                   0
Total Rebounds                       0
Assists                              0
Steals                               0
Blocks                   

In [11]:
players_df4 = players_df3.fillna(.000)
players_df4

,Player,Position,Age,Team,Game,Games Started,Minutes Played,Field Goals,Field Goals Attempted,Field Goal Percentage,...,Free Throw Percentage,Offensive Rebounds,Defensive Rebounds,Total Rebounds,Assists,Steals,Blocks,Turnovers,Personal Fouls,Points
0,Arron Afflalo,SG,24,DEN,82,75,2221,272,585,.465,...,.735,59,193,252,138,46,30,74,225,724
1,Alexis Ajinça,C,21,CHA,6,0,30,5,10,.500,...,.000,1,3,4,0,1,1,2,5,10
2,LaMarcus Aldridge,PF,24,POR,78,78,2922,579,1169,.495,...,.757,192,435,627,160,67,48,104,231,1393
3,Joe Alexander,SF,23,CHI,8,0,29,1,6,.167,...,.667,2,3,5,2,1,1,0,9,4
4,Malik Allen,PF,31,DEN,51,3,456,46,116,.397,...,.923,36,46,82,16,11,5,22,68,105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8976,Thaddeus Young,PF,34,TOR,54,9,795,108,198,.545,...,.692,71,95,166,75,54,5,42,88,240
8977,Trae Young,PG,24,ATL,73,73,2541,597,1390,.429,...,.886,56,161,217,741,80,9,300,104,1914
8978,Omer Yurtseven,C,24,MIA,9,0,83,16,27,.593,...,.833,8,15,23,2,2,2,4,16,40
8979,Cody Zeller,C,30,MIA,15,2,217,37,59,.627,...,.686,25,39,64,10,3,4,14,33,98


In [12]:
nan_count = players_df4.isna().sum()
print(nan_count )

Player                             0
Position                           0
Age                                0
Team                               0
Game                               0
Games Started                      0
Minutes Played                     0
Field Goals                        0
Field Goals Attempted              0
Field Goal Percentage              0
Three Point Field Goals            0
Three Pointers Attempted           0
Three Point Percentage             0
Two Point Field Goals              0
Two Pointers Attempted             0
Two Point Percentage               0
Effective Field Goal Percentage    0
Free Throws Made                   0
Free Throws Attempted              0
Free Throw Percentage              0
Offensive Rebounds                 0
Defensive Rebounds                 0
Total Rebounds                     0
Assists                            0
Steals                             0
Blocks                             0
Turnovers                          0
P

We can see there is no longer any missing values. I decided to change all of the missing values to .000 because the only missing values were because they had 0 attempts in that stat so it would be undefined for percentages.

# Ethical Implications

Changing the undefined values to 0 could be considered an ethical implication due to the fact that technically their stats are not equivalent to the other people who have .000 because they never did actually do it. All the other data that I cleaned have no ethical implications.